In [24]:
import numpy as np
import pandas as pd

# jupyter配置
pd.options.display.max_rows=10000 #Notebook 的一个cell的显示行数
pd.options.display.max_columns=10000#Notebook 的一个cell的显示列数

In [15]:
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

In [12]:
train_data[train_data["product_id"] == 154533]

,user_id,product_id,product_name,rating,votes,helpful_votes,ID
0,1813,154533,Beautiful Thing,5,10,8,0
145997,765,154533,Beautiful Thing,4,0,0,145997
151303,1255,154533,Beautiful Thing,5,4,4,151303
237732,1560,154533,Beautiful Thing,4,6,5,237732
275139,85,154533,Beautiful Thing,4,2,2,275139
288065,506,154533,Beautiful Thing,5,6,6,288065
365036,923,154533,Beautiful Thing,5,9,8,365036
500212,1080,154533,Beautiful Thing,5,1,1,500212
681987,1916,154533,Beautiful Thing,4,2,2,681987


In [19]:
train_data['user_id'].value_counts()
train_data['votes'].value_counts()

votes
1      96872
0      91758
2      87807
3      75892
4      60134
       ...  
263        1
329        1
354        1
274        1
436        1
Name: count, Length: 348, dtype: int64

In [27]:
train_data['helpful_votes'].value_counts()

helpful_votes
0       146769
1       130552
2        96563
3        72808
4        53515
5        41277
6        31603
7        25549
8        19730
9        16079
10       14276
11       11776
12        9588
13        8433
14        7270
15        6087
16        5197
17        4544
18        4156
19        3418
20        3069
21        2898
22        2332
23        2048
24        1908
25        1721
26        1647
27        1501
28        1297
30        1226
29        1208
31        1028
32         886
33         857
34         840
35         664
37         659
36         629
39         577
38         507
40         486
41         469
42         414
43         386
45         376
44         376
48         262
46         260
50         255
47         240
49         239
51         222
52         218
56         208
53         200
54         194
55         175
57         175
58         161
59         142
60         133
62         126
61         121
64         120
65         113
63         

In [13]:
train_data['product_id'].value_counts()

product_id
212937    275
212359    267
192681    255
213993    253
212370    251
         ... 
85657       1
115804      1
103529      1
173043      1
171557      1
Name: count, Length: 201325, dtype: int64

In [14]:
train_data['rating'].value_counts()

rating
5    416231
4    185193
3     79870
2     35446
1     29149
Name: count, dtype: int64

In [17]:
test_data[test_data["user_id"] == 1916]

,ID,user_id,product_id,product_name
0,0,1916,185507,Maria [Australia]
12,12,1916,192801,Dinosaur (2-Disc Collector's Edition)
18,18,1916,70224,When Hitler Stole Pink Rabbit
19,19,1916,152813,Junior
23,23,1916,19551,NRSV Ministry/Pew Bible
...,...,...,...,...
223545,223545,1916,187413,Millennium
223546,223546,1916,186727,The Slim Shady LP
223547,223547,1916,38308,Hammer of the Gods
223548,223548,1916,214704,"Godzilla, King of the Monsters"


## user feature

In [ ]:
# 计算用户特征
user_features = train_data.groupby('user_id').agg({
    'rating': ['mean', 'count'],
    'votes': ['mean', 'max'],
    'helpful_votes': ['mean', 'max']
}).reset_index()
user_features.columns = ['user_id', 'user_rating_mean', 'user_rating_count', 'user_votes_mean', 'user_votes_max', 
                         'user_helpful_votes_mean', 'user_helpful_votes_max']
user_features

## product feature 计算产品特征

In [28]:
# 计算产品特征
product_features = train_data.groupby('product_id').agg({
    'rating': ['mean', 'count'],
    'votes': ['mean', 'max'],
    'helpful_votes': ['mean', 'max']
}).reset_index()
product_features.columns = ['product_id', 'product_rating_mean', 'product_rating_count', 'product_votes_mean', 'product_votes_max',
                            'product_helpful_votes_mean', 'product_helpful_votes_max']
product_features

,product_id,product_rating_mean,product_rating_count,product_votes_mean,product_votes_max,product_helpful_votes_mean,product_helpful_votes_max
0,0,4.400000,5,8.000000,34,7.400000,32
1,1,4.500000,4,15.250000,23,13.250000,22
2,3,4.076923,26,6.846154,56,4.923077,51
3,4,4.000000,1,7.000000,7,4.000000,4
4,5,5.000000,1,0.000000,0,0.000000,0
...,...,...,...,...,...,...,...
201320,219854,3.250000,4,6.250000,12,2.000000,4
201321,219855,4.000000,4,4.500000,11,3.000000,7
201322,219856,4.888889,9,10.666667,32,10.000000,29
201323,219857,3.750000,4,4.750000,7,3.750000,7


In [29]:
train_data = train_data.merge(user_features, on='user_id').merge(product_features, on='product_id')
train_data

,user_id,product_id,product_name,rating,votes,helpful_votes,ID,user_rating_mean,user_rating_count,user_votes_mean,user_votes_max,user_helpful_votes_mean,user_helpful_votes_max,product_rating_mean,product_rating_count,product_votes_mean,product_votes_max,product_helpful_votes_mean,product_helpful_votes_max
0,1813,154533,Beautiful Thing,5,10,8,0,4.251799,556,7.611511,88,5.462230,70,4.555556,9,4.444444,10,4.0,8
1,1916,154533,Beautiful Thing,4,2,2,681987,4.458700,112483,7.315808,681,5.298009,657,4.555556,9,4.444444,10,4.0,8
2,506,154533,Beautiful Thing,5,6,6,288065,4.785417,480,2.343750,20,1.685417,16,4.555556,9,4.444444,10,4.0,8
3,923,154533,Beautiful Thing,5,9,8,365036,4.299320,147,11.204082,94,10.224490,92,4.555556,9,4.444444,10,4.0,8
4,1560,154533,Beautiful Thing,4,6,5,237732,4.143437,739,3.657645,51,2.652233,46,4.555556,9,4.444444,10,4.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745884,1850,140202,Britney Spears,3,4,3,437598,4.128788,132,2.977273,14,2.265152,13,3.000000,1,4.000000,4,3.0,3
745885,1850,79222,Get Wise! Mastering Writing Skills,5,0,0,449862,4.128788,132,2.977273,14,2.265152,13,5.000000,1,0.000000,0,0.0,0
745886,1850,1185,Soul Survivors: The Official Autobiography of ...,4,2,1,567969,4.128788,132,2.977273,14,2.265152,13,4.000000,1,2.000000,2,1.0,1
745887,1850,71558,Artistry on Ice: Figure Skating Skills and Style,5,2,2,666395,4.128788,132,2.977273,14,2.265152,13,5.000000,1,2.000000,2,2.0,2


In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# 定义特征和目标变量
features = ['user_rating_mean', 'user_votes_mean', 'product_rating_mean', 'product_votes_mean']
X = train_data[features]
y = train_data['rating']

# 划分训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练模型
model = XGBRegressor()
model.fit(X_train, y_train)

# 验证模型
y_pred = model.predict(X_val)
print("MSE:", mean_squared_error(y_val, y_pred))
